# Prepare the EDA Environment

In [1]:
import sys
import os
import types
import warnings
import glob
import regex as re
import netCDF4 as nc
import rasterio
import numpy as np
import xarray as xr
import hvplot.xarray
import holoviews as hv
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf
import ipywidgets as widgets
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
import chardet
import pyperclip
from geopandas import GeoDataFrame
from shapely.geometry import Point
from bokeh.io import output_notebook, show
from bokeh.resources import INLINE
from rasterio.transform import from_origin
from rasterstats import zonal_stats
from shapely.geometry import LineString
from matplotlib.path import Path
from matplotlib.colors import Normalize
from netCDF4 import Dataset
from pyproj import CRS
from IPython.display import display
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from _windbreaks_helpers import *

In [2]:
warnings.filterwarnings('ignore')

## Set data source variables

In [3]:
# Path to the directory
directory = 'Data'

# Check if the directory exists
if os.path.isdir(directory):
    src_dir = directory
else:
    src_dir = None

print(src_dir)
%store src_dir

Data
Stored 'src_dir' (str)


## Set the extents of the Area of Interest (AOI)

In [4]:
# Project extents
extent_coords = {'min_lat': 36.9314320, 'max_lat': 43.1345169,
                 'min_lon': -104.2232889, 'max_lon': -94.5426297}
%store extent_coords

Stored 'extent_coords' (dict)


In [5]:
# Use Jupyter magik to list all variables and function loaded in the interactive workspace
%whos

Variable                     Type             Data/Info
-------------------------------------------------------
CRS                          type             <class 'pyproj.crs.crs.CRS'>
Dataset                      type             <class 'netCDF4._netCDF4.Dataset'>
GeoDataFrame                 type             <class 'geopandas.geodataframe.GeoDataFrame'>
INLINE                       Resources        Resources(mode='inline')
LATITUDE_FORMATTER           FuncFormatter    <matplotlib.ticker.FuncFo<...>ct at 0x000001EDA24EF410>
LONGITUDE_FORMATTER          FuncFormatter    <matplotlib.ticker.FuncFo<...>ct at 0x000001EDA24EF390>
LineString                   type             <class 'shapely.geometry.linestring.LineString'>
Normalize                    type             <class 'matplotlib.colors.Normalize'>
Path                         type             <class 'matplotlib.path.Path'>
Point                        type             <class 'shapely.geometry.point.Point'>
ccrs                    

## Load basemaps and boundary files for AOI

In [6]:
# Load the county boundary shapefile
ksne_fn = os.path.join(src_dir, 'GIS_files/NE_KS_county.shp')
ksne_data = gpd.read_file(ksne_fn)

# Get the CRS
crs = CRS(ksne_data.crs)

# Name of the CRS
print('Name:', crs.name)
# EPSG of the CRS
if crs.to_epsg():
    print('EPSG:', crs.to_epsg())
else:
    print('No EPSG found for this CRS')
# Unit of the CRS
print('Unit:', crs.axis_info[0].unit_name)

# Preview Data
ksne_data.head(6)

Name: NAD83
EPSG: 4269
Unit: degree


,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,20,053,00484996,0500000US20053,20053,Ellsworth,06,1853331063,19969761,"POLYGON ((-98.48554 38.78395, -98.48561 38.794..."
1,20,095,00485012,0500000US20095,20095,Kingman,06,2236100137,8536908,"POLYGON ((-98.46489 37.64530, -98.46481 37.703..."
2,20,135,00485031,0500000US20135,20135,Ness,06,2783562240,667491,"POLYGON ((-100.24774 38.65441, -100.24720 38.6..."
3,20,199,00485060,0500000US20199,20199,Wallace,06,2366348284,140936,"POLYGON ((-102.04719 39.13315, -102.02582 39.1..."
4,31,091,00835867,0500000US31091,31091,Hooker,06,1867796137,882994,"POLYGON ((-101.42656 42.09219, -101.41220 42.0..."
5,31,167,00835905,0500000US31167,31167,Stanton,06,1107500701,8789497,"POLYGON ((-97.36840 42.09092, -97.30991 42.090..."


In [7]:
ksne_data = ksne_data.to_crs('EPSG:15914')
# Get the CRS
crs = CRS(ksne_data.crs)

# Name of the CRS
print('Name:', crs.name)
# EPSG of the CRS
if crs.to_epsg():
    print('EPSG:', crs.to_epsg())
else:
    print('No EPSG found for this CRS')
# Unit of the CRS
print('Unit:', crs.axis_info[0].unit_name)
# get column names with their index
print('\n', 'ksne_data columns:', '\n')

output_string = ''

for i, col_name in enumerate(ksne_data.columns):
    dtype = ksne_data[col_name].dtype
    output_string += f'Index: {i}, Column Name: {col_name}, Data Type: {dtype}\n'

pyperclip.copy('ksne_data:\n' + output_string)
print(output_string)

CRSError: Invalid projection: EPSG:15914: (Internal Proj Error: proj_create: crs not found)

# Examine Storm Event Data

## - Examine Storm Event csv's

In [12]:
modified_se_data.to_csv(os.path.join(src_dir, 'Storm_event/StormEvents_all.csv'))

In [13]:
modified_se_data = modified_se_data.drop(columns='EVENT_ID_1').rename(
    columns={'BEGIN_YEAR': 'BEGIN_YRMO', 'END_YEARMO': 'END_YRMO', 'CZ_FIPS': 'COUNTY_FIP', 'CZ_NAME': 'COUNTY'})

In [14]:
modified_se_data.to_csv(os.path.join(src_dir, 'Storm_event/StormEvents_alltest.csv'))

In [15]:
# list all columns, in the order you want them
column_order = modified_se_data.columns.tolist()

# remove 'COUNTY' and 'COUNTY_FIPS' from their current location and
# insert them right after 'STATE' (index 12) and 'STATE_FIPS' (index 13)
column_order.remove('COUNTY')
column_order.remove('COUNTY_FIP')
column_order.insert(14, 'COUNTY')
column_order.insert(15, 'COUNTY_FIP')

# reindex the dataframe
modified_se_data = modified_se_data[column_order]
print('\n', 'modified_se_data columns:', '\n')
# get column names with their index
output_string = ''

for i, col_name in enumerate(original_se_data.columns):
    dtype = original_se_data[col_name].dtype
    output_string += f'Index: {i}, Column Name: {col_name}, Data Type: {dtype}\n'

pyperclip.copy('modified_se_data:\n' + output_string)
print(output_string)


 modified_se_data columns: 

Index: 0, Column Name: OID_, Data Type: int64
Index: 1, Column Name: EVENT_ID, Data Type: int64
Index: 2, Column Name: BEGIN_LAT, Data Type: float64
Index: 3, Column Name: BEGIN_LON, Data Type: float64
Index: 4, Column Name: END_LAT, Data Type: float64
Index: 5, Column Name: END_LON, Data Type: float64
Index: 6, Column Name: Shape_Leng, Data Type: float64
Index: 7, Column Name: Field1, Data Type: int64
Index: 8, Column Name: EVENT_ID_1, Data Type: int64
Index: 9, Column Name: BEGIN_YEAR, Data Type: int64
Index: 10, Column Name: BEGIN_DAY, Data Type: int64
Index: 11, Column Name: END_YEARMO, Data Type: int64
Index: 12, Column Name: END_DAY, Data Type: int64
Index: 13, Column Name: STATE, Data Type: object
Index: 14, Column Name: STATE_FIPS, Data Type: int64
Index: 15, Column Name: YEAR, Data Type: int64
Index: 16, Column Name: MONTH_NAME, Data Type: object
Index: 17, Column Name: EVENT_TYPE, Data Type: object
Index: 18, Column Name: DAMAGE_CRO, Data Type: o

In [16]:
# Get the CRS
crs = CRS(modified_se_data.crs)

# Name of the CRS
print('Name:', crs.name)
# EPSG of the CRS
if crs.to_epsg():
    print('EPSG:', crs.to_epsg())
else:
    print('No EPSG found for this CRS')
# Unit of the CRS
print('Unit:', crs.axis_info[0].unit_name)


Name: NAD83
EPSG: 4269
Unit: degree


In [17]:
# Reproject / transform to BLM zone 14N (US survey foot)
modified_se_data = modified_se_data.to_crs('EPSG:15914')
# Get the CRS
crs = CRS(modified_se_data.crs)

# Name of the CRS
print('Name:', crs.name)
# EPSG of the CRS
if crs.to_epsg():
    print('EPSG:', crs.to_epsg())
else:
    print('No EPSG found for this CRS')
# Unit of the CRS
print('Unit:', crs.axis_info[0].unit_name)
print("Unique Years: ", modified_se_data['YEAR'].unique())
print("Unique Counties: ", modified_se_data['COUNTY'].unique())

Name: NAD83(2011) / Kansas South (ftUS)
EPSG: 6469
Unit: US survey foot
Unique Years:  [2011 2012 2013 2014 2015 2016 2017 2018 2019]
Unique Counties:  ['MARION' 'BUTLER' 'HARRISON' 'RANDOLPH' 'COOPER' 'SHERIDAN' 'CLARK'
 'GRAY' 'DENT' 'OKMULGEE' 'NOWATA' 'CREEK' 'CRAIG' 'COLUMBIA' 'GRAHAM'
 'HODGEMAN' 'FORD' 'KIOWA' 'PRATT' 'PAWNEE' 'COWLEY' 'GENTRY' 'MERCER'
 'HOLT' 'ANDREW' 'MACON' 'SULLIVAN' 'DAVIESS' 'LINN' 'CHASE' 'RUSSELL'
 'RICE' 'MCPHERSON' 'DEUEL' 'UNION' 'GRUNDY' 'NODAWAY' 'ELK' 'ALLEN'
 'MONTGOMERY' 'CHERRY' 'HARVEY' 'TULSA' 'MILLER' 'HEMPSTEAD' 'ATOKA'
 'KEITH' 'CRITTENDEN' 'THOMAS' 'KIT CARSON' 'KEARNY' 'CALDWELL' 'ADAIR'
 'WOODSON' 'CHAUTAUQUA' 'GRANT' 'COMANCHE' 'LANE' 'FINNEY' 'SEDGWICK'
 'GREENE' 'PITTSBURG' 'MORGAN' 'MCCURTAIN' 'LE FLORE' 'LARAMIE'
 'SCOTTS BLUFF' 'LINCOLN' 'LITTLE RIVER' 'CUSTER' 'ROCK' 'NEVADA' 'DUNDY'
 'YUMA' 'HAMILTON' 'JACKSON' 'MEADE' 'DE KALB' 'CLAY' 'PLATTE' 'WRIGHT'
 'GAGE' 'BOX BUTTE' 'ANTELOPE' 'PONTOTOC' 'NEOSHO' 'LABETTE' 'CASS'
 'DUNKLI

In [18]:
# Save modified data as a new Shapefile
modified_shp_file = os.path.join(src_dir, 'Storm_event/modified_storm_events.shp')
modified_se_data.to_file(modified_shp_file)

In [19]:
# Filter the GeoDataFrame based on extent and save as a new shapefile
output_shp_file = os.path.join(src_dir, 'Storm_event/filtered_storm_events.shp')
gdf_to_shp(modified_se_data, output_shp_file, extent_coords)

# Load the filtered data
filtered_se_data = gpd.read_file(output_shp_file)
filtered_se_data.crs

<Projected CRS: EPSG:6469>
Name: NAD83(2011) / Kansas South (ftUS)
Axis Info [cartesian]:
- X[east]: Easting (US survey foot)
- Y[north]: Northing (US survey foot)
Area of Use:
- name: United States (USA) - Kansas - counties of Allen; Anderson; Barber; Barton; Bourbon; Butler; Chase; Chautauqua; Cherokee; Clark; Coffey; Comanche; Cowley; Crawford; Edwards; Elk; Finney; Ford; Franklin; Grant; Gray; Greeley; Greenwood; Hamilton; Harper; Harvey; Haskell; Hodgeman; Kearny; Kingman; Kiowa; Labette; Lane; Linn; Lyon; Marion; McPherson; Meade; Miami; Montgomery; Morton; Neosho; Ness; Osage; Pawnee; Pratt; Reno; Rice; Rush; Scott; Sedgwick; Seward; Stafford; Stanton; Stevens; Sumner; Wichita; Wilson; Woodson.
- bounds: (-102.05, 36.99, -94.6, 38.88)
Coordinate Operation:
- name: SPCS83 Kansas South zone (US Survey feet)
- method: Lambert Conic Conformal (2SP)
Datum: NAD83 (National Spatial Reference System 2011)
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [20]:
print(type(output_shp_file))
print(output_shp_file)
print(modified_se_data['geometry'].head())


<class 'str'>
Data\Storm_event/filtered_storm_events.shp
0    None
1    None
2    None
3    None
4    None
Name: geometry, dtype: geometry


In [21]:
# Specify the path to the input geodataframe
input_gdf = filtered_se_data
# Specify the buffer distance as a float or int
buf_dist = 3
# or you can specify a str if you want to buffer by meters or feet instesd of kilometers and and miles, respectively
# buf_dist = '3000ft'
buf_dist_str = str(buf_dist).replace('.', '_')
# Specify the path to the output shapefile
output_path = os.path.join(src_dir, f'Storm_event/storm_line_{buf_dist_str}_buf.shp')

# Create a buffered GeoDataFrame
# Note: For Geographic Coordinate Systems, provide buffer distance in kilometers.
# For BLM zone 14N (US survey foot) Coordinate Systems, provide buffer distance in miles.
buffered_storm_events = gdf_buf(input_gdf, buf_dist, output_path)

In [22]:
# Group by 'county' and 'END_YEARMO' and aggregate
se_gdf = filtered_se_data #.groupby(['county', 'END_YEARMO']).sum().reset_index()

min_value = se_gdf['BEGIN_YRMO'].min()
max_value = se_gdf['END_YRMO'].max()

print('Minimum value in BEGIN_YRMO column:', min_value)
print('Maximum value in END_YRMO column:', max_value)


Minimum value in BEGIN_YRMO column: 201102
Maximum value in END_YRMO column: 201912


In [23]:
# Load the storm line data from a shapefile
buf_dist_str = '3'
stormbuf_gdf = gpd.read_file(os.path.join(src_dir, f'Storm_event/storm_line_{buf_dist_str}_buf.shp'))
# Perform the spatial join (intersection)
intersect_gdf = gpd.sjoin(ksne_data, stormbuf_gdf, how='inner', op='intersects')
intersect_gdf.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,...,STATE,STATE_FIPS,COUNTY,COUNTY_FIP,YEAR,MONTH_NAME,EVENT_TYPE,DAMAGE_CRO,MAGNITUDE,MAGNITUDE_
0,20,037,00484988,0500000US20037,20037,Crawford,06,1527498199,13574680,"POLYGON ((2301012.649 1655071.499, 2300977.621...",...,KANSAS,20,CHEROKEE,21,2012,April,Thunderstorm Wind,0.00K,50.0,EG
2,20,021,00484980,0500000US20021,20021,Cherokee,06,1521791027,8951473,"POLYGON ((2307788.521 1575772.445, 2331036.824...",...,KANSAS,20,CHEROKEE,21,2012,April,Thunderstorm Wind,0.00K,50.0,EG
0,20,037,00484988,0500000US20037,20037,Crawford,06,1527498199,13574680,"POLYGON ((2301012.649 1655071.499, 2300977.621...",...,KANSAS,20,CHEROKEE,21,2019,May,Tornado,0.00K,0.0,None
2,20,021,00484980,0500000US20021,20021,Cherokee,06,1521791027,8951473,"POLYGON ((2307788.521 1575772.445, 2331036.824...",...,KANSAS,20,CHEROKEE,21,2019,May,Tornado,0.00K,0.0,None
0,20,037,00484988,0500000US20037,20037,Crawford,06,1527498199,13574680,"POLYGON ((2301012.649 1655071.499, 2300977.621...",...,KANSAS,20,CRAWFORD,37,2019,May,Tornado,0.00K,0.0,None


In [24]:
# get column names with their index
print('\n', 'stormbuf_gdf columns:', '\n')
output_string = ''

for i, col_name in enumerate(stormbuf_gdf.columns):
    dtype = stormbuf_gdf[col_name].dtype
    output_string += f'Index: {i}, Column Name: {col_name}, Data Type: {dtype}\n'

pyperclip.copy('stormbuf_gdf:\n' + output_string)
print(output_string)


 stormbuf_gdf columns: 

Index: 0, Column Name: EVENT_ID, Data Type: int64
Index: 1, Column Name: OID_, Data Type: int64
Index: 2, Column Name: BEGIN_LAT, Data Type: float64
Index: 3, Column Name: BEGIN_LON, Data Type: float64
Index: 4, Column Name: END_LAT, Data Type: float64
Index: 5, Column Name: END_LON, Data Type: float64
Index: 6, Column Name: Shape_Leng, Data Type: float64
Index: 7, Column Name: Field1, Data Type: int64
Index: 8, Column Name: BEGIN_YRMO, Data Type: int64
Index: 9, Column Name: BEGIN_DAY, Data Type: int64
Index: 10, Column Name: END_YRMO, Data Type: int64
Index: 11, Column Name: END_DAY, Data Type: int64
Index: 12, Column Name: STATE, Data Type: object
Index: 13, Column Name: STATE_FIPS, Data Type: int64
Index: 14, Column Name: COUNTY, Data Type: object
Index: 15, Column Name: COUNTY_FIP, Data Type: int64
Index: 16, Column Name: YEAR, Data Type: int64
Index: 17, Column Name: MONTH_NAME, Data Type: object
Index: 18, Column Name: EVENT_TYPE, Data Type: object
Inde

In [25]:
print('ksne_data', ksne_data.crs)
print('se_gdf', se_gdf.crs)
print('stormbuf_gdf', stormbuf_gdf.crs)

sixco_data EPSG:6469
se_gdf EPSG:6469
stormbuf_gdf EPSG:6469


In [26]:
# get column names with their index
print('\n', 'intersect_gdf columns:', '\n')
output_string = ''

for i, col_name in enumerate(intersect_gdf.columns):
    dtype = intersect_gdf[col_name].dtype
    output_string += f'Index: {i}, Column Name: {col_name}, Data Type: {dtype}\n'

pyperclip.copy('intersect_gdf:\n' + output_string)
print(output_string)


 intersect_gdf columns: 

Index: 0, Column Name: STATEFP, Data Type: object
Index: 1, Column Name: COUNTYFP, Data Type: object
Index: 2, Column Name: COUNTYNS, Data Type: object
Index: 3, Column Name: AFFGEOID, Data Type: object
Index: 4, Column Name: GEOID, Data Type: object
Index: 5, Column Name: NAME, Data Type: object
Index: 6, Column Name: LSAD, Data Type: object
Index: 7, Column Name: ALAND, Data Type: int64
Index: 8, Column Name: AWATER, Data Type: int64
Index: 9, Column Name: geometry, Data Type: geometry
Index: 10, Column Name: index_right, Data Type: int64
Index: 11, Column Name: EVENT_ID, Data Type: int64
Index: 12, Column Name: OID_, Data Type: int64
Index: 13, Column Name: BEGIN_LAT, Data Type: float64
Index: 14, Column Name: BEGIN_LON, Data Type: float64
Index: 15, Column Name: END_LAT, Data Type: float64
Index: 16, Column Name: END_LON, Data Type: float64
Index: 17, Column Name: Shape_Leng, Data Type: float64
Index: 18, Column Name: Field1, Data Type: int64
Index: 19, C

In [27]:
%whos

Variable                     Type             Data/Info
-------------------------------------------------------
CRS                          type             <class 'pyproj.crs.crs.CRS'>
Dataset                      type             <class 'netCDF4._netCDF4.Dataset'>
GeoDataFrame                 type             <class 'geopandas.geodataframe.GeoDataFrame'>
INLINE                       Resources        Resources(mode='inline')
LATITUDE_FORMATTER           FuncFormatter    <matplotlib.ticker.FuncFo<...>ct at 0x000001C65004EA50>
LONGITUDE_FORMATTER          FuncFormatter    <matplotlib.ticker.FuncFo<...>ct at 0x000001C65004E990>
LineString                   type             <class 'shapely.geometry.linestring.LineString'>
Normalize                    type             <class 'matplotlib.colors.Normalize'>
Path                         type             <class 'matplotlib.path.Path'>
Point                        type             <class 'shapely.geometry.point.Point'>
buf_dist                

In [28]:
# get column names with their index
print('\n', 'se_gdf columns:', '\n')
output_string = ''

for i, col_name in enumerate(se_gdf.columns):
    dtype = se_gdf[col_name].dtype
    output_string += f'Index: {i}, Column Name: {col_name}, Data Type: {dtype}\n'

pyperclip.copy('se_gdf columns:\n' + output_string)
print(output_string)


 se_gdf columns: 

Index: 0, Column Name: EVENT_ID, Data Type: int64
Index: 1, Column Name: OID_, Data Type: int64
Index: 2, Column Name: BEGIN_LAT, Data Type: float64
Index: 3, Column Name: BEGIN_LON, Data Type: float64
Index: 4, Column Name: END_LAT, Data Type: float64
Index: 5, Column Name: END_LON, Data Type: float64
Index: 6, Column Name: Shape_Leng, Data Type: float64
Index: 7, Column Name: Field1, Data Type: int64
Index: 8, Column Name: BEGIN_YRMO, Data Type: int64
Index: 9, Column Name: BEGIN_DAY, Data Type: int64
Index: 10, Column Name: END_YRMO, Data Type: int64
Index: 11, Column Name: END_DAY, Data Type: int64
Index: 12, Column Name: STATE, Data Type: object
Index: 13, Column Name: STATE_FIPS, Data Type: int64
Index: 14, Column Name: COUNTY, Data Type: object
Index: 15, Column Name: COUNTY_FIP, Data Type: int64
Index: 16, Column Name: YEAR, Data Type: int64
Index: 17, Column Name: MONTH_NAME, Data Type: object
Index: 18, Column Name: EVENT_TYPE, Data Type: object
Index: 19,

In [29]:
se_gdf.head()

,EVENT_ID,OID_,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,Shape_Leng,Field1,BEGIN_YRMO,BEGIN_DAY,...,STATE_FIPS,COUNTY,COUNTY_FIP,YEAR,MONTH_NAME,EVENT_TYPE,DAMAGE_CRO,MAGNITUDE,MAGNITUDE_,geometry
0,320977,0,37.15,-95.78,37.15,-95.78,0.0,802,201106,20,...,20,MONTGOMERY,125,2011,June,Thunderstorm Wind,0.00K,52.0,EG,None
1,325620,0,37.53,-95.36,37.53,-95.36,0.0,1843,201106,10,...,20,NEOSHO,133,2011,June,Thunderstorm Wind,None,70.0,EG,None
2,325622,0,37.37,-95.27,37.37,-95.27,0.0,1845,201106,10,...,20,LABETTE,99,2011,June,Thunderstorm Wind,None,57.0,MG,None
3,305195,0,37.10,-95.57,37.10,-95.57,0.0,2333,201105,23,...,20,MONTGOMERY,125,2011,May,Thunderstorm Wind,None,54.0,MG,None
4,305200,0,37.05,-95.23,37.05,-95.23,0.0,2665,201105,23,...,20,LABETTE,99,2011,May,Thunderstorm Wind,None,52.0,EG,None


In [30]:
print("Unique Years: ", se_gdf['YEAR'].unique())
print("Unique Counties: ", se_gdf['COUNTY'].unique())


Unique Years:  [2011 2012 2013 2014 2015 2016 2017 2018 2019]
Unique Counties:  ['MONTGOMERY' 'NEOSHO' 'LABETTE' 'WILSON' 'CHEROKEE' 'CRAWFORD' 'BOURBON'
 'OTTAWA' 'JASPER' 'BARTON']


In [31]:
se_sum_gdf = se_gdf.groupby(['YEAR', 'STATE', 'COUNTY'])['EVENT_ID'].size()
print(se_sum_gdf)
print('\n', 'se_sum_gdf index:', '\n')
output_string = ''

for i, index_name in enumerate(se_sum_gdf.index.names):
    output_string += f'Index Level: {i}, Index Name: {index_name}\n'

pyperclip.copy('se_sum_gdf index:\n' + output_string)
print(output_string)


YEAR  STATE     COUNTY    
2011  KANSAS    BOURBON        2
                CHEROKEE      12
                CRAWFORD       9
                LABETTE       11
                MONTGOMERY    11
                NEOSHO        10
                WILSON         2
2012  KANSAS    CHEROKEE      19
                CRAWFORD      23
                LABETTE       10
                MONTGOMERY     8
                NEOSHO         5
                WILSON         1
2013  KANSAS    CHEROKEE       8
                CRAWFORD       4
                LABETTE        3
                MONTGOMERY    11
                NEOSHO         1
                WILSON         3
2014  KANSAS    CHEROKEE      10
                CRAWFORD       2
                LABETTE        5
                MONTGOMERY    10
                NEOSHO         2
      MISSOURI  JASPER         2
      OKLAHOMA  OTTAWA         1
2015  KANSAS    CHEROKEE       9
                CRAWFORD       4
                LABETTE       14
                

In [32]:
se_sum_gdf = se_sum_gdf.reset_index()

se_sum_gdf.head()

,YEAR,STATE,COUNTY,EVENT_ID
0,2011,KANSAS,BOURBON,2
1,2011,KANSAS,CHEROKEE,12
2,2011,KANSAS,CRAWFORD,9
3,2011,KANSAS,LABETTE,11
4,2011,KANSAS,MONTGOMERY,11


# Store notebook variables for use in other notebooks in the project.

In [144]:
%store src_dir
%store extent_coords
%store crop_data
%store se_gdf
%store stormbuf_gdf
%store ksne_data

Stored 'src_dir' (str)
Stored 'extent_coords' (dict)
Stored 'crop_data' (DataFrame)
Stored 'se_gdf' (GeoDataFrame)
Stored 'stormbuf_gdf' (GeoDataFrame)
Stored 'sixco_data' (GeoDataFrame)
